In [20]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

setup geometry

In [21]:
geo = SplineGeometry()
geo.AddRectangle( (-3,-2), (3, 2), bcs = ("top", "out", "bot", "in"))
geo.AddCircle ( (0, 0), r=0.5, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.2))
mesh.Curve(3);
#Draw(mesh)

setup FEM space

In [22]:
# viscosity
nu = 0.01
# Order of spaces
k = 2
# H1 vs VectorH1 -> vector field?!
V = VectorH1(mesh,order=k, dirichlet="top|bot|cyl|in|out")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q]) # X = [V,V,Q] (without VectorH1)

setup bilinear form
velocityfield u and pressurefield p

In [23]:
u,p = X.TrialFunction()
v,q = X.TestFunction()

a = BilinearForm(X)
a += (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a.Assemble()

setup boundary conditions

In [24]:
gfu = GridFunction(X)
#'gfp = GridFunction(X)
# setup flow condition
uinf = 0.001
uin = CoefficientFunction((uinf,0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("in|top|bot|out"))

velocity = CoefficientFunction(gfu.components[0])
scene = Draw(velocity, mesh, "vel")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

solve equation

In [25]:
def solveStokes():
    res = gfu.vec.CreateVector()
    res.data = -a.mat * gfu.vec
    inv = a.mat.Inverse(X.FreeDofs())
    gfu.vec.data += inv * res
    scene.Redraw()
solveStokes()

Drag/ "dissipated energy" [should be working]

$J(\Omega ) = \frac{1}{2} \int_\Omega Du : Du dx$ 

In [26]:
solveStokes()
vel = gfu.components[0] #gfu.vec.data
drag = Integrate(0.5*InnerProduct(Grad(vel),Grad(vel)),mesh)
print(drag)

# 1.0997749905576028e-05 - r=0.5
# 0.00043084429187575745 - r=1.5

1.09978300562245e-05


In [27]:
def calcDrag(gfu):
    vel = gfu.components[0]
    return 0.5*InnerProduct(grad(vel),grad(vel))*dx
'''
eigentlich brauchen wir das gar nicht, weil das ist ja kein side-constraint sondern die haupt shape funktion
'''

'\neigentlich brauchen wir das gar nicht, weil das ist ja kein side-constraint sondern die haupt shape funktion\n'

### get surface area of mesh [done]
(without ball), should stay constant

In [28]:
surf = Integrate(1,mesh)
print(surf)
def calcSurf():
    return Integrate(1,mesh)
surf_test = GridFunction(X)
surf_test.components[0].Set(CoefficientFunction((1,1)))
def calcSurf2(gfu):
    return surf_test*dx
print(Integrate(surf_test.components[0],mesh)) # works \o/

23.21459270293146
 23.2146
 23.2146



### try for barycenter <br>
$vol(\Omega) = \int_{\Omega} 1 \,dx \in \mathbb{R}$ <br>
$bc^\Omega = \frac{1}{vol(\Omega)}\int_{\Omega}x\,dx \in \mathbb{R}^d$

In [29]:
def calc_bc(gfu):
    surf_test2 = GridFunction(X)
    surf_test2.components[0].Set(CoefficientFunction((1,1)))
    bc_test = GridFunction(X)
    bc_test.components[0].Set(CoefficientFunction((x,y)))
    r = surf_test2*bc_test
    return surf_test2*dx+bc_test*dx

In [30]:
def Cost(gfu):
    return calc_bc(gfu) + calcSurf2(gfu) + calcDrag(gfu)# + calcDrag(gfu) # + calcSurf2() + calcDrag(gfu)

## Adjoint equation try?
maybe we dont even need this??

* J hängt von u ab, u von omega
* dJ nach u -> "in Vektorfeldrichtung" -> für jedes Vektorfeld ableiten
* stattdessen adjoint -> 1x u und 1x omega
* adjoint
* Lagrangian du -> adjoint

In [31]:
V2 = VectorH1(mesh,order=k, dirichlet="top|bot|in|out|cyl")
gfp = GridFunction(X)
r, w = V2.TnT()

fadjoint = LinearForm(X)
#adjoint_diff = CoefficientFunction(gfu)
fadjoint += -1*(Cost(gfu)).Diff(gfu,w)

In [32]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(X.FreeDofs()).T * rhs
    gfp.vec.data += update

In [33]:
fadjoint.Assemble()
SolveAdjointEquation()
#print(gfp.vec.data)
adjoint_sol = CoefficientFunction(gfp.components[0])
Draw (adjoint_sol, mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [34]:
a.Assemble()
solveStokes()

________________________________

### creation of gfset
function that will perturb our mesh

In [35]:
# Test and trial functions for shape derivate -> do we even need this?
VEC = H1(mesh, order=2, dim=2)
PHI, PSI = VEC.TnT()
# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
mesh.SetDeformation(gfset)
sceneSet = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

### Shape derivative $\mathrm{d}J(\Omega)(\mathbf{X}) = \int_{\Omega} \mathbf{S}_1:D \mathbf{X} dx$
$S_1 = \biggl(\frac{1}{2}Du : Du - p div(u) \biggr) I_2 + Du^T p - Du^T Du.$ <br>
$\mathrm{u}$ and $p$ are the solutions to the stokesflow problem<br>
we use u = gfu[0], p = gfu[1]

# Auto shapediff try

In [37]:
def Equation(u,v):
    return 0.5*(InnerProduct(grad(u.components[0]),grad(u.components[0])))*dx

Lagrangian = Cost(gfu) + Equation(gfu,gfp)

## this basically just copy paste

In [20]:
def SolveDeformationEquationAuto():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmegaAuto.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [ ]:
b.Assemble()
dJOmegaAuto.Assemble()
#SolveDeformationEquationAuto()
#Draw(-gfX, mesh, "-gfX")

In [1]:
print('Cost at initial design', Integrate (Cost(gfu), mesh))
gfset.Set((0,0))
scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec

NameError: name 'Integrate' is not defined

In [ ]:
scene = Draw(gfset)
mesh.SetDeformation(gfset)
scene.Redraw()

a.Assemble()
fadjoint.Assemble()
solveStokes()
print('Cost at new design', Integrate (Cost(gfu), mesh))

In [ ]:
#reset the design
gfset.Set((0,0))
mesh.SetDeformation(gfset)

#check if initial value of cost function 0.000486578350214552 is recovered
a.Assemble()
fadjoint.Assemble()
solveStokes()
print('Cost at new design', Integrate (Cost(gfu), mesh))

# general idea: of iteration

* solve solution
* deform
* calc cost after deform
* look how cost changed -> armijo rule
* abstieg: -gradient²

In [ ]:
scene = Draw(gfset)

In [ ]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
a.Assemble()
fadjoint.Assemble()
solveStokes()

LineSearch = False

iter_max = 600
Jold = Integrate(Cost(gfu), mesh)
converged = False

# try parts of loop
mesh.SetDeformation(gfset)
scene.Redraw()

print('cost at iteration', k, ': ', Jold)

a.Assemble()
solveStokes()

fadjoint.Assemble()
SolveAdjointEquation()

#b.Assemble()
#dJOmega.Assemble()
#SolveDeformationEquation()
    
'''
# input("Press enter to start optimization")
for k in range(iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    print('cost at iteration', k, ': ', Jold)
    
    a.Assemble()
    solveStokes()
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmega.Assemble()
    SolveDeformationEquation()

    scale = 0.01 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    
    Jnew = Integrate(Cost(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            #input('a')
            scale = scale / 2
            print("scale = ", scale)
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            a.Assemble()
            fstate.Assemble()
            SolveStateEquation()
            Jnew = Integrate(Cost(gfu), mesh)
    
    if converged==True:
        print("No more descent can be found")
        break
    Jold = Jnew

    Redraw(blocking=True)
'''